# 二重矢板の予測（回帰モデル）

In [ ]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
import japanize_matplotlib
# LightGBM
import lightgbm as lgb
import optuna
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder

## 1.CSVファイルの読み込み

In [ ]:
#データの読み込み
data_folder = input("データファイルのあるフォルダまでのパス")
data_folder = data_folder.rstrip()
data_folder = data_folder.replace("\\", "/") + "/"
    
file = data_folder + "input20001_30000_input+error.csv"
# file1 = data_folder + "train_data1.csv"
# file2 = data_folder + "train_labels.csv"

df = pd.read_csv(file,encoding="shift-jis")


# df1 = pd.read_csv(file1,encoding="cp932")
# df2 = pd.read_csv(file2,encoding="cp932")

# df = pd.concat([df1,df2],axis = 1)

In [ ]:
pd.set_option('display.max_columns',50)
df

## ラベルエンコーディング

In [ ]:
df.dtypes

In [ ]:
def label_en(la,df):
    label = df[la]
    le_embarked = LabelEncoder()
    le_embarked.fit(label)
    df[la] =  le_embarked.transform(label)
    df[la] = df[la].astype("category")
    return

#タイプ3:0，タイプ4:1，タイプ5:2
label_en('在来地盤区分',df)
label_en('中詰め区分',df)
label_en('矢板材料',df)
label_en('矢板型',df)
label_en('腹起し番号',df)
label_en('引張材材料番号',df)
label_en('腹起し材質',df)
# label_en('せん断変形破壊_label',df)
# label_en('滑動、支持力_label',df)
# label_en('根入れ部の安定_label',df)
# label_en('遮水効果_label',df)
# label_en('矢板の耐力照査_label',df)
# label_en('タイロッドの耐力照査_label',df)
# label_en('腹起しの耐力照査_label',df)

In [ ]:
df.dtypes

In [ ]:
pd.set_option('display.max_columns',50)
df

In [ ]:
pd.set_option('display.max_columns',50)
df.head()

In [ ]:
pd.set_option('display.max_columns',40)
df.head()

In [ ]:
safety_label =  ["せん断変形破壊","滑動、支持力","根入れ部の安定","遮水効果","矢板の耐力照査","タイロッドの耐力照査","腹起しの耐力照査","根入れ深さ"]
# safety_label =  ["せん断変形破壊_label","滑動、支持力_label","根入れ部の安定_label","遮水効果_label","矢板の耐力照査_label","タイロッドの耐力照査_label","腹起しの耐力照査_label","根入れ深さ"]
df = df.drop([i for i in safety_label],axis=1)

In [ ]:
df

## 相関を調べる

In [ ]:
# 目的変数の設定
pur = "矢板の全長"

In [ ]:
# sns.set(rc = {'figure.figsize':(15,8)})
sns.set(font='Yu Gothic')
plt.figure(figsize=(20,20),dpi=200)
p=sns.heatmap(df.corr(),square=True, vmax=1, vmin=-1, center=0,cmap='coolwarm')

In [ ]:
sns.set(font='Yu Gothic',rc = {'figure.figsize':(20,20)})
sns.heatmap(df.corr()[[pur]].sort_values(by=pur, ascending=False)[1:],cmap='coolwarm', annot=True)

In [ ]:
df

In [ ]:
#目的変数の分布
def plot_histgram(x, title=None, x_label=None):
    fig,ax = plt.subplots()
    sns.set(font='Yu Gothic',rc = {'figure.figsize':(12,6)})

    plt.figure(figsize=(8,8),dpi=500)
    sns.histplot(x, kde = False, ax=ax)
    fig.suptitle(title)
    ax.set_xlabel(x_label)

    fig.show()

plot_histgram(df[pur], title="目的変数の分布", x_label=pur)

In [ ]:
df[pur].describe()

In [ ]:
#目的変数と各特徴量の関係
def plt_scatters(X, y, title = None,nrows=2,ncols=2):
    # sns.set(rc = {'figure.figsize':(15,8)})
    sns.set(font='Yu Gothic',rc = {'figure.figsize':(30,30)})

    cols = X.columns
    plt.figure(figsize=(4,4),dpi=200)
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols)

    for ax,c in zip(axes.ravel(),cols):
        sns.scatterplot(X[c], y , ci=None, ax=ax)
        ax.set(ylabel = y.name)
    

    fig.suptitle(title)
    fig.show()

xx = df.drop([pur], axis = 1)
plt_scatters(
    df[xx.columns],
    df[pur],
    title = "目的変数と各特徴量との関係",
    nrows = 7,
    ncols = 5
)

In [ ]:
plt_scatters(
    df[['左右壁体高さ','堤外側水位_常時']],
    df['中詰土天端高さ'],
    title = "目的変数と各特徴量との関係",
    nrows = 2,
    ncols = 1
)

## モデルの作成

### データの分割

In [ ]:
# データの分割
# 全体の20%をテストデータに設定

from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3, random_state = 3)

In [ ]:
# Stratified K Foldでデータを分割
from imblearn.under_sampling import RandomUnderSampler
# 目的変数と説明変数に分ける
x_train = train.drop([pur], axis = 1) # 予測対象以外を説明変数に設定
y_train = train.loc[:,pur]

### 交差検証

In [ ]:
# Stratified K Foldでデータを分割
from imblearn.under_sampling import RandomUnderSampler
# 目的変数と説明変数に分ける
X = train.drop([pur], axis = 1) # 予測対象以外を説明変数に設定
y = train.loc[:,pur]

# データの分割
# ライブラリのインポート
from sklearn.model_selection import KFold

fold = KFold(n_splits=5, shuffle=True, random_state=3) # データを5分割する
kf = fold.split(X, y)
kf_cv = list(kf)

In [ ]:
for i, (idx_train, idx_val) in enumerate(kf_cv):
    print(f'fold {i}')
    print(idx_train)
    print(idx_val)
    print('=='*30)
    print(len(idx_train), len(idx_val)) #5分割しているのでデータ数が1:4になるか確認する
    print('=='*30)

## LightGBM

In [ ]:
# from xgboost.callback import early_stop
from sklearn import metrics # 正解率を出すためのライブラリ
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,StratifiedKFold,cross_val_score

### ハイパーパラメータチューニング

In [ ]:
params_base = {
    "boosting_type": "gbdt",
    'objective': 'regression_l1', 
    'metric': 'mean_absolute_error',
    'n_estimators': 1000,
    "subsample":0.7,
    'subsample_freq': 1,
    "seed": 123,
    "importance_type":"gain"
}

In [ ]:
def objective(trial, df_X, df_y):

    params_tuning = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
        "num_leaves": trial.suggest_int("num_leaves", 8, 256),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 5, 200),
        "min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 5, 200),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e2, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e2, log=True)
    }

    params_tuning.update(params_base)


    model = lgb.LGBMRegressor(verbosity=2,
                             n_jobs=-1,
                             random_state=42,
                             **params_tuning)

    # 交差検証
    scores = cross_val_score(
        model, df_X, df_y, scoring='AUC', cv=5)
    rmse = np.sqrt(-scores)
    score_mean = np.mean(rmse)

    return score_mean


In [ ]:
#optuna.create_study()でoptuna.studyインスタンスを作る。
study = optuna.create_study()

#studyインスタンスのoptimize()に作った関数を渡して最適化する。
study.optimize(lambda trial: objective(trial,X,y),n_trials=200, timeout=300)


In [ ]:
params_best = study.best_params
params_best.update(params_base)
display(params_best)

In [ ]:
#スコアを見る
print(study.best_params)    

In [ ]:
print(study.best_value)

In [ ]:
# xgb_params = {
#     'objective': 'reg:squarederror',  # 回帰問題
#     'eval_metric': 'rmse',       # 学習用の指標
# }

In [ ]:
# xgb_params['max_depth'] = study.best_params['max_depth']
# xgb_params['min_child_weight'] = study.best_params['min_child_weight']
# xgb_params['gamma'] = study.best_params['gamma']
# xgb_params['subsample'] = study.best_params['subsample']
# xgb_params['colsample_bytree'] = study.best_params['colsample_bytree']
# xgb_params['learning_rate'] = study.best_params['learning_rate']
# # xgb_params['reg_alpha'] = study.best_params['reg_alpha']
# # xgb_params['reg_lambda'] = study.best_params['reg_lambda']

### 学習開始

In [ ]:
from sklearn.metrics import average_precision_score,mean_absolute_error
import shap

def fit_lgb(X, y, cv, params: dict=None):
    models = []
    ma = []
    r = []
    # acc = []
    oof = np.zeros(len(X))
    # oof_classfication = np.zeros(len(X))

    if params is None:
        params = {}

    for i, (idx_train, idx_val) in enumerate(kf_cv):
        X_train, y_train = X.iloc[idx_train], y.iloc[idx_train] # 学習用の説明変数と目的変数の呼び出し
        X_val, y_val = X.iloc[idx_val], y.iloc[idx_val]

        reg = lgb.LGBMRegressor(**params)
        model = reg.fit(X_train, y_train,
                        eval_set=[(X_val, y_val)],  
                        early_stopping_rounds=20,
                        verbose = 2)

        pred = model.predict(X_val)
        oof[idx_val] = pred

        models.append(model)

        shap.initjs()

        explainer = shap.TreeExplainer(model)
        shap_values = explainer(X_train) 

        shap.plots.bar(shap_values=shap_values,max_display=10)
        shap.plots.beeswarm(shap_values,max_display=10)
        
        print('r2_train:',reg.score(X_train, y_train))
        print('r2_val:',reg.score(X_val, y_val))
        print('MAE_val: ',mean_absolute_error(y_val, pred))
        # print('Acc_val: ',metrics.accuracy_score(y_val, pred))
        ma.append(metrics.mean_absolute_error(y_val, pred))
        # acc.append(metrics.accuracy_score(y_val, pred))
        r.append(reg.score(X_val, y_val))

    print(f'平均のMAE：{np.mean(ma)}')
    print(f'平均のR2：{np.mean(r)}')
    # print(f'平均のAcc：{np.mean(acc)}')
    return oof, models

In [ ]:
oof,models = fit_lgb(X, y, kf_cv, params_best)

### テストデータ

In [ ]:
X_test = test.drop([pur], axis=1)
y_test = test[pur]

In [ ]:
# from sklearn.metrics import average_precision_score,mean_absolute_error
from sklearn.metrics import mean_squared_error, mean_absolute_error

def inference_lgb(models):
    # testデータに対して推論を行う
    X_test = test.drop([pur], axis=1)
    y_test = test[pur]

    pred_test = np.zeros((5,len(y_test))) # 320×6の2次元配列を作成
    r = []
    # acc = []

    for i,model in enumerate(models):
        pred_test[i] = model.predict(X_test)/5
        r.append(model.score(X_test, y_test))

        explainer = shap.TreeExplainer(model) #TreeExplainerでも多分変わらないが・・
        shap_values = explainer(X_test) #これで全確認データの数字を一気に入手

        shap.plots.bar(shap_values=shap_values,max_display=10)
        shap.plots.beeswarm(shap_values,max_display=10)
        
        # acc.append(metrics.accuracy_score(y_test, pred_test))
    pred_test = np.sum(pred_test, axis=0) 

    print('MAE_test: ',mean_absolute_error(y_test, pred_test))
    print('r2_test_average:',np.mean(r))
    # print('acc_average:',np.mean(acc))

    return pred_test,X_test,y_test

In [ ]:
pred_test,X_test,y_test = inference_lgb(models)

### 全体の確認

In [ ]:
train["predict"] = oof
train

In [ ]:
test["predict"] = pred_test
test

In [ ]:
df_pred = pd.concat([train,test])
df_pred = df_pred.sort_index()
df_pred

In [ ]:
# データの予測値と正解値とのずれ
df_pred["Error"] = df_pred["predict"] - df_pred[pur]
df_pred.sort_values('Error')
print(df_pred["Error"].describe())


In [ ]:
#過大評価
df_error_plus = df_pred[df_pred["Error"]>1]
df_error_plus

In [ ]:
#過小評価
df_error_minus = df_pred[df_pred["Error"]<-1]
df_error_minus

In [ ]:
#x軸が予測値，y軸が結果
import matplotlib.pyplot as plt
import numpy as np
plt.scatter(df_error_minus["predict"],df_error_minus[pur], alpha = 0.5)
plt.scatter(df_error_plus["predict"],df_error_plus[pur], alpha = 0.5)
plt.plot(np.linspace(0, 40, 40), np.linspace(0, 40, 40), "red")
plt.show()

In [ ]:
#過大評価
plt.hist(df_error_plus["堤体幅"],alpha=0.5,color="blue")
#過小評価
plt.hist(df_error_minus["堤体幅"],alpha=0.5,color="orange")

## 予測結果の可視化

In [ ]:
#x軸が予測値，y軸が結果
import matplotlib.pyplot as plt
import numpy as np
sns.set(font='Yu Gothic',rc = {'figure.figsize':(10,10)})
plt.scatter(train["predict"],train[pur], alpha = 0.5,color = "blue",label = "Train",s=2)
plt.scatter(test["predict"],test[pur], alpha = 0.5,color = "orange",label = "test",s=2)
plt.plot(np.linspace(0, 20, 20), np.linspace(0, 20, 20), "red")
plt.xlabel("予測値")
plt.ylabel("真値")
plt.legend()
plt.show()

###  特徴量重要度

In [ ]:
# 5つのモデルで重要度が出てくるので箱ひげ図にします、
def plot_importance(model, X):
    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df["feature_importance"] = model.feature_importances_
        _df["column"] = X.columns
        _df["fold"] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], 
                                          axis=0, ignore_index=True)

    order = feature_importance_df.groupby("column").sum()[["feature_importance"]].sort_values("feature_importance", ascending=False).index[:50]

    fig, ax = plt.subplots(figsize=(8, max(6, len(order) * .25)))
    sns.boxenplot(data=feature_importance_df, 
                  x="feature_importance", 
                  y="column", 
                  order=order, 
                  ax=ax, 
                  palette=None,  
                  orient="h")
    ax.tick_params(axis="x")
    ax.set_title("Feature Importance")
    ax.grid()
    fig.tight_layout()
    return fig, ax

fig, ax = plot_importance(models, X)

### shap値個々の値についてしらべる

In [ ]:
def shap_part(val):
    X_predict = df_pred.drop([pur,"predict"], axis=1)
    y_predict = df_pred[pur] 

    for model in models:
        explainer = shap.TreeExplainer(model = model,data=X_predict,feature_perturbation="interventional")
        shap_values = explainer(X_predict)
        shap.plots.waterfall(shap_values=shap_values[val],max_display=20)

In [ ]:
#引数にはデータの何番目の値の詳細を見たいかをいれる．
#モデルごとにwaterfall図が出る．モデルによって異なる．
#f(x):モデルの予測値
#E[f(x)]:モデル予測値全体の平均


# shap_part(1)

In [ ]:
# 5つのモデルで重要度が出てくるので箱ひげ図にします、
def plot_importance(model, val):
    X_predict = df_pred.drop([pur,"predict"], axis=1)
    y_predict = df_pred[pur] 

    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        explainer = shap.TreeExplainer(model = model,data=X_predict,feature_perturbation="interventional")
        shap_values = explainer(X_predict)
        _df = pd.DataFrame()
        _df["shap_values"] = shap_values[val]
        _df["column"] = X_predict.columns
        _df["fold"] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], 
                                          axis=0, ignore_index=True)

    order = feature_importance_df.groupby("column").sum()[["shap_values"]].sort_values("shap_values", ascending=False).index[:50]

    fig, ax = plt.subplots(figsize=(8, max(6, len(order) * .25)))
    sns.boxenplot(data=feature_importance_df, 
                  x="shap_values", 
                  y="column", 
                  order=order, 
                  ax=ax, 
                  palette=None,  
                  orient="h")
    ax.tick_params(axis="x")
    ax.set_title("shap_values")
    ax.grid()
    fig.tight_layout()
    return fig, ax

fig, ax = plot_importance(models, 1)